In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
# 배치 크기 * 채널 * 높이(height) * 너비(width)의 크기의 텐서를 선언
inputs = torch.Tensor(1, 1, 28, 28)
print('텐서의 크기 : {}'.format(inputs.shape))

텐서의 크기 : torch.Size([1, 1, 28, 28])


In [12]:
conv1 = nn.Conv2d(1, 32, 3, padding = 1)

In [4]:
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [6]:
conv2 = nn.Conv2d(32, 64, kernel_size = (3,3), stride=(1, 1), padding=(1,1))

In [7]:
pool = nn.MaxPool2d(2)

In [13]:
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [14]:
out = pool(out)

In [17]:
print(out.shape)

torch.Size([1, 32, 14, 14])


In [18]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [19]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [20]:
out.size(0)

1

In [21]:
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


In [22]:
fc = nn.Linear(3136, 10)
out = fc(out)
print(out.shape)

torch.Size([1, 10])


In [23]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init 

In [24]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)

if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [25]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [27]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train = True,
                          transform=transforms.ToTensor(),
                          download=True)

Mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)   

100%|██████████| 9912422/9912422 [00:00<00:00, 350418145.11it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 132244207.23it/s]


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 128967671.73it/s]


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 18317816.12it/s]


Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [28]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [33]:
class CNN(torch.nn.Module):

  def __init__(self):
    super(CNN, self).__init__()
    # 첫번째층
    # ImgIn shape = (?, 28, 28, 1)
    # conv -> (?, 28, 28, 32)
    # pool -> (?, 14, 14, 32)

    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2))
      # 두번째층
      # ImgIn shape=(?, 14, 14, 32)
      # Conv -> (?, 14, 14, 64)
      # pool -> (?, 7, 7, 64)
      
    self.layer2 = torch.nn.Sequential(
          torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
          torch.nn.ReLU(),
          torch.nn.MaxPool2d(kernel_size=2, stride=2))

    self.fc = torch.nn.Linear(7*7*64, 10, bias=True)

    torch.nn.init.xavier_uniform_(self.fc.weight)
  

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.view(out.size(0), -1)
    out = self.fc(out)
    return out

In [34]:
model = CNN().to(device)

In [35]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [36]:
total_batch = len(data_loader)
print(total_batch)

600


In [37]:
for epoch in range(training_epochs):
  avg_cost = 0
  for X, Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost/total_batch
  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))


[Epoch:    1] cost = 0.22562106
[Epoch:    2] cost = 0.0631106719
[Epoch:    3] cost = 0.0463027246
[Epoch:    4] cost = 0.0374040492
[Epoch:    5] cost = 0.0312901363
[Epoch:    6] cost = 0.0261751674
[Epoch:    7] cost = 0.0217915345
[Epoch:    8] cost = 0.0183159597
[Epoch:    9] cost = 0.0162470695
[Epoch:   10] cost = 0.0131644458
[Epoch:   11] cost = 0.0101902885
[Epoch:   12] cost = 0.00970787276
[Epoch:   13] cost = 0.00842072535
[Epoch:   14] cost = 0.00698324759
[Epoch:   15] cost = 0.00676915795


In [40]:
with torch.no_grad():
  X_test = Mnist_test.test_data.view((len(Mnist_test), 1, 28, 28)).float().to(device)
  Y_test = Mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1)==Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

Accuracy: 0.9878999590873718


/usr/local/lib/python3.9/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
